In [ ]:
import pandas as pd
import numpy as np

df = pd.read_parquet("../data/train.parquet")

In [ ]:
df["length"] = df["session_mask"].map(lambda x: len(x))

In [ ]:
import multiprocessing
df = [elem for elem in df.groupby('id')]

In [ ]:
from tqdm import tqdm

cores = multiprocessing.cpu_count()

def concatenate(vectors):
    elements = [elem for elem in vectors]
    return np.concatenate(elements, axis=None)

def append_dataframe(df):
    id = df[0]
    df = df[1]
    
    df = df.sort_values("timestamp")
    history = concatenate(df["history"].values)
    timestamp = concatenate(df["timestamp"].values)
    session = concatenate(df["session"].values)
    session_mask = concatenate(df["session_mask"].values)
    user_mask = concatenate(df["user_mask"].values)
    
    return pd.DataFrame({
        "id": [id],
        "history": [history],
        "timestamp": [timestamp],
        "session": [session],
        "session_mask": [session_mask],
        "user_mask": [user_mask],
        "length": np.sum(df["length"].values)
    })

with multiprocessing.Pool(cores) as p:
    df = list(tqdm(p.imap(append_dataframe, df), total=len(df)))

In [ ]:
df = pd.concat(df)

In [ ]:
df

In [ ]:
df.to_parquet("../data/masked.parquet")

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_parquet("../data/brunch/session.parquet")

In [ ]:
import json
from tqdm import tqdm

with open("../data/dictionary.json") as fp:
    dictionary = json.load(fp)
    
def make_label(x):
    session = x["history"]
    session = [dictionary[elem] for elem in session if elem in dictionary]
    x["session"] = session
    return x

tqdm.pandas()
df = df.progress_apply(make_label, axis=1)

In [ ]:
with open("../data/brunch/predict/dev.users") as fp:
    dev = [elem[0:-1] for elem in fp]
with open("../data/brunch/predict/test.users") as fp:
    test = [elem[0:-1] for elem in fp]

In [ ]:
df_dev = df[df.id.isin(dev)]

In [ ]:
df_dev_grouped = df_dev.groupby("id")

In [ ]:
from tqdm import tqdm

printed = False

def get_length(x):
    id = x[0]
    frame = x[1]
    global printed
    
    session = frame.session
    lengths = [len(elem) for elem in session]
    return pd.DataFrame({
        "id": [id],
        "length": [np.sum(lengths)]
    })

df_dev_grouped = [elem for elem in df_dev_grouped]
sampled = df_dev_grouped[0:100]
df_list = []
for elem in tqdm(sampled):
    df_list.append(get_length(elem))


In [ ]:
df_dev = pd.concat(df_list)

In [ ]:
df_dev.sort_values("length")

In [ ]:
df[df.id == "#009bca89575df8ed68a302c1ceaf7da4"]

In [ ]:
df

In [ ]:
df = [row for index, row in df.iterrows()]

In [ ]:
df[0]

In [ ]:
def user_parallel_process(frame):    
    session = frame.session
    if len(session) < 2:
        return -1
    frame["session_input"] = session[0:-1]
    frame["session_output"] = session[1:]
    
    session = session[0:-1]
    session_length = len(session)
    # generating session mask
    session_mask = [1.0] * (session_length - 1)
    session_mask = [0.0] + session_mask
    # generating user mask
    user_mask = [0.0] * (session_length - 1)
    user_mask = user_mask + [1.0]
    frame["session_mask"] = session_mask
    frame["user_mask"] = user_mask
    
    return frame

cores = multiprocessing.cpu_count()
with multiprocessing.Pool(cores) as p:
    df = list(tqdm(p.imap(user_parallel_process, df), total=len(df)))

In [ ]:
import json

with open("../data/dictionary.json") as fp:
    dictionary = json.load(fp)

In [ ]:
keys = []
values = []
for key, value in dictionary.items():
    keys.append(key)
    values.append(value)
    
import pandas as pd

df = pd.DataFrame({
    "id" : keys,
    "pos" : values
})

In [ ]:
df.to_parquet("../data/brunch/dataframe_dictionary.parquet")

In [1]:
import pyarrow.parquet as pq
import json
import pandas as pd

with open("../data/dictionary.json") as fp:
    dictionary = json.load(fp)
dataset = pq.ParquetDataset("../data/brunch/train")
table = dataset.read()
df = table.to_pandas()

In [2]:
from tqdm import tqdm
tqdm.pandas()

In [3]:
import numpy as np

def numpy_fill(arr):
    '''Solution provided by Divakar.'''
    mask = np.isnan(arr)
    idx = np.where(~mask,np.arange(mask.shape[1]),0)
    np.maximum.accumulate(idx,axis=1, out=idx)
    out = arr[np.arange(idx.shape[0])[:,None], idx]
    return out

def forward_fill(record):
    
    pos = record["pos"]
    session_input = record["session_input"]
    session_output = record["session_output"]
    
    if len(pos) == 0:
        record["trainable"] = False
        return record
    
    if len(session_input) == 0 and len(session_output) == 0:
        session_input = [float(pos[0])]
        session_output = [float(pos[0])]
    
    input_nans = np.isnan(session_input)
    output_nans = np.isnan(session_output)
    
    if all(input_nans):
        record["trainable"] = False
        return record
    if np.isnan(session_input[0]):
        session_input[0] = float(len(dictionary))
    session_input = numpy_fill(np.array([session_input]))[0]
    record["session_input"] = session_input
    
    if all(output_nans):
        record["trainable"] = False
        return record
    if np.isnan(session_output[0]):
        session_output[0] = session_input[1]
    session_output = numpy_fill(np.array([session_output]))[0]
    record["session_output"] = session_output
    record["trainable"] = True
    
    return record

In [4]:
df = df.progress_apply(forward_fill, axis=1)

100%|██████████| 3493934/3493934 [1:07:30<00:00, 862.69it/s] 


In [5]:
df.to_parquet("../data/brunch/train.parquet")

In [6]:
df[df.id == "#00104b6ef7bea05a3264ea0ab197fba9"]

,id,session,history,idx,timestamp,pos,session_input,session_output,session_mask,user_mask,trainable
561926,#00104b6ef7bea05a3264ea0ab197fba9,1342,"[@wootaiyoung_19, @snobberys_109, @tenbody_902...","[5, 3, 0, 4, 2, 1]","[2018-11-25T22:00:00.000000000, 2018-11-25T22:...","[60128, 355501, 38319, 568640]","[38319.0, 38319.0, 38319.0, 355501.0, 568640.0]","[38319.0, 38319.0, 355501.0, 568640.0, 60128.0]","[0.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 1.0]",True


In [11]:
sample_df.to_parquet("../data/brunch/sample_train.parquet")

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import json

from tqdm import tqdm

with open("../data/dictionary.json") as fp:
    dictionary = json.load(fp)

df = pd.read_parquet("../data/brunch/train.parquet")

/Users/jeasungpark/Repository/hgru4rec/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jeasungpark/Repository/hgru4rec/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jeasungpark/Repository/hgru4rec/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jeasungpark/

In [2]:
df[df.id == "#00104b6ef7bea05a3264ea0ab197fba9"]

,id,session,history,idx,timestamp,pos,session_input,session_output,session_mask,user_mask,trainable
561926,#00104b6ef7bea05a3264ea0ab197fba9,1342,"[@wootaiyoung_19, @snobberys_109, @tenbody_902...","[5, 3, 0, 4, 2, 1]","[2018-11-25T22:00:00.000, 2018-11-25T22:00:00....","[60128, 355501, 38319, 568640]","[38319.0, 38319.0, 38319.0, 355501.0, 568640.0]","[38319.0, 38319.0, 355501.0, 568640.0, 60128.0]","[0.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 1.0]",True


In [3]:
df_trainable = df[df.trainable == True]
df_trainable = df_trainable.groupby("id")

In [4]:
input_list = []
label_list = []
mask_list = []

idx = 0
max_length = 30
dictionary_length = len(dictionary)

for idx, (key, frame) in tqdm(enumerate(df_trainable), total=len(df_trainable)):
        frame = frame.sort_values("session")
    
        session_input = np.concatenate(frame.session_input.values, axis=None)
        session_output = np.concatenate(frame.session_output.values, axis=None)
        session_mask = np.concatenate(frame.session_mask.values, axis=None)
        user_mask = np.concatenate(frame.user_mask.values, axis=None)
            
        message = "At least one of the dimension doesn't match in the input."
        assert len(session_input) == len(session_output), message
        assert len(session_output) == len(session_mask), message
        assert len(session_mask) == len(user_mask), message
        
        if len(session_input) > 30:
            continue
            
        inputs = [session_input, session_mask, user_mask]
        inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, 
                                                               maxlen=max_length, 
                                                               padding="post")
        label = tf.keras.preprocessing.sequence.pad_sequences([session_output], 
                                                              maxlen=max_length, 
                                                              value=np.float64(dictionary_length), 
                                                              padding="post")

        mask = [1.0] * len(session_input)
        mask = tf.keras.preprocessing.sequence.pad_sequences([mask],
                                                           maxlen=max_length,
                                                           value = 0.0,
                                                           padding="post")
        
        input_list.append(inputs)
        label_list.append(label)
        mask_list.append(mask)
        
        if key == "#00104b6ef7bea05a3264ea0ab197fba9":
            print(label)
        
inputs = np.array(input_list)
label = np.array(label_list)
mask = np.array(mask_list)

  0%|          | 197/301930 [00:06<244:59:40,  2.92s/it]

[[ 38319  38319 355501 568640  60128 642190 642190 642190 642190 642190
  642190 642190 642190 642190 642190 642190 642190 642190 642190 642190
  642190 642190 642190 642190 642190 642190 642190 642190 642190 642190]]


100%|██████████| 301930/301930 [05:06<00:00, 986.68it/s] 


In [9]:
np.save("../data/brunch/train", inputs)
np.save("../data/brunch/label", label)
np.save("../data/brunch/mask", mask)